In [22]:
import os,sys,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

In [2]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_lower_woParam_0616_768.bin') # sent2vec model
all_df = pd.read_csv('./data/tree-rep-profiles-partial/process2family_df.csv')
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

26

In [3]:
encode_dict = {}
for i,v in enumerate(api_li):
    encode_dict[v] = i+1
pickle.dump(file=open('data/tree-rep-profiles-partial/encode_dict.pkl','wb'),obj=encode_dict)
encode_dict
# encode_dict = pickle.load(open('data/tree-rep-profiles-partial/encode_dict.pkl','rb'))
# encode_dict

{'loadlibrary': 1,
 'createprocess': 2,
 'openprocess': 3,
 'exitprocess': 4,
 'terminateprocess': 5,
 'winexec': 6,
 'createremotethread': 7,
 'createthread': 8,
 'copyfile': 9,
 'createfile': 10,
 'deletefile': 11,
 'regsetvalue': 12,
 'regcreatekey': 13,
 'regdeletekey': 14,
 'regdeletevalue': 15,
 'regqueryvalue': 16,
 'regenumvalue': 17,
 'winhttpconnect': 18,
 'winhttpopen': 19,
 'winhttpopenrequest': 20,
 'winhttpreaddata': 21,
 'winhttpsendrequest': 22,
 'internetopen': 23,
 'internetconnect': 24,
 'httpsendrequest': 25,
 'geturlcacheentryinfo': 26}

## stage 1 data
* from one2many take all except valid test pid
* 兩種stage 1 data (main 、 pids)

In [4]:
train_pids_df,valid_pids_df,test_pids_df = pickle.load(file=open('./data/tree-rep-profiles-partial/pids_3df.pkl','rb'))
train_main_df,valid_main_df,test_main_df = pickle.load(file=open('./data/tree-rep-profiles-partial/main_3df.pkl','rb'))
print(len(train_pids_df),len(valid_pids_df),len(test_pids_df))
len(train_main_df) , len(valid_main_df) , len(test_main_df)

6251 347 347


(4853, 269, 270)

In [53]:
'''root_dir_dir = './data/tree-rep-profiles-partial/normal/'
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir +fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for profile in hl_list:
            with open(profile,encoding='ISO 8859-1') as f:
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
            lines = lines.splitlines()
            for line in lines:
                temp = re.sub(dil," ",line)#.lower()) #lower小寫?
                temp = temp.split(" ")
                temp = list(filter(None, temp))
                temp = ' '.join(temp)
                if temp.startswith('WinH'):
                    print(profile,func_name)
            '''


100%|██████████| 184/184 [00:11<00:00,  8.80it/s]

*  stage 1: 2 kind of train

In [5]:
def Sent2vec_emb(root_dir,valid_pids_df,test_pids_df,max_length,model=model,dim=768,encode_dict=encode_dict):
    # root_dir = './data/tree-rep-profiles_one2many/'
    samples_pid_aliases = []
    processed_pid = []
    rasmma_dir = next(os.walk(root_dir))[1]
    samples_pid_valid = valid_pids_df.index.tolist()
    samples_pid_test = test_pids_df.index.tolist()
    # max_length = 347
    # dim = 768
    sent_pad = [0]*dim
    # encode_dict = encode_dict_lower
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    hl_rep = []
    sen2vec_length_normalize = []
    for rasmma in rasmma_dir:
        fam_dir = next(os.walk(root_dir + rasmma))[1]
        for fam in tqdm(fam_dir):
            tree_dir = next(os.walk(root_dir + rasmma +'/' +fam))[1]
            for tree in tree_dir:
                in_directory = root_dir +rasmma +'/'+ fam +  '/' + tree + '/'
                hl_list = next(os.walk(in_directory))[2]
                hl_list = [os.path.join(in_directory, f) for f in hl_list]
                hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
                hl_list_final = hl_list
                for hl in hl_list:
                    name = hl.split('/')[-1].split('.')[0]
                    if name in samples_pid_valid: #移除在valid
                        hl_list_final.remove(hl)
                    if name in samples_pid_test: #移除在test
                        hl_list_final.remove(hl)
                for hl_f in hl_list_final:
                    profile_emb = []
                    func_name_emb = []
                    byterep = hl_f.replace('.profile','_byterep.pickle') #拿出hl_f所對應的byterep
                    rep = pickle.load(open(byterep,'rb')) #list type
                    if len(rep) < max_length: #Y1
                        for _ in range(max_length-len(rep)):
                            rep.append(0)
                    with open(hl_f,encoding='ISO 8859-1') as f: #X2
                        lines = f.read()
                    lines = re.sub(r'[^\x00-\x7F]+','', lines)
                    lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                    lines = lines.splitlines()
                    if len(lines) > max_length:
                        continue
                    for line in lines:
                        temp = re.sub(dil," ",line.lower()) # lower? 跟先前一致
                        temp = temp.split(" ")
                        temp = list(filter(None, temp))
                        temp = ' '.join(temp)
                        func_name = temp.split(' ')[0] # X1
                        if func_name not in api_li:
                            print('=ERROR:=',profile,'=>',temp)
                        func_name_emb.append(encode_dict[func_name]) # encode証術數數字
                        emb = model.embed_sentence(temp)
                        emb = emb[0]
                        profile_emb.append(emb) #放入embdding，句子=>profile
                    sen2vec_length_normalize.append(len(lines))
                    if len(lines) < max_length:
                        for _ in range(max_length-len(lines)):
                            profile_emb.append(sent_pad)
                    hl_rep.append(rep) #Y1
                    all_profiles.append(profile_emb) #X2
                    all_api_name.append(func_name_emb) #X1
    #             all_byterep.extend(rep_list)
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0) #X1
    return np.array(all_api_name) , np.array(all_profiles) , np.array(hl_rep) ,sen2vec_length_normalize

In [ ]:
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" #等號、底線被保留，注意要跟先前一致
train_api , train_emb , train_rep, train_len_norm = Sent2vec_emb('./data/tree-rep-profiles_one2many/',
                                                 valid_pids_df,test_pids_df,347)
print(train_api.shape , train_emb.shape , train_rep.shape)
train_api_main , train_emb_main , train_rep_main, train_len_norm_main = Sent2vec_emb('./data/tree-rep-profiles_one2many/',
                                                               valid_main_df, test_main_df,347)
print(train_api_main.shape , train_emb_main.shape , train_rep_main.shape)


  0%|          | 0/184 [00:00<?, ?it/s]

(17263, 347) (17263, 347, 768) (17263, 347)


 22%|██▏       | 41/184 [00:52<04:00,  1.68s/it]

* stage 1: 2 kind of valid and test

In [ ]:
def Sent2vec_emb_dev(root_dir,samples_pid_dev,max_length,model=model,dim=768,encode_dict=encode_dict):

# root_dir = './data/tree-rep-profiles-partial/normal/'
# samples_pid_dev = valid_pids_df.index.tolist()
# max_length = 347
# dim = 768

    sent_pad = [0]*dim
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    hl_rep = []
    sen2vec_length_normalize = []

    for pid in tqdm(samples_pid_dev):
        profile_paths = glob.glob(root_dir + '*/*/'+pid+'.profile')
        byterep_paths = glob.glob(root_dir + '*/*/'+pid+'_byterep.pickle')
        assert len(profile_paths) == len(byterep_paths)
        for hl_f in profile_paths:
            profile_emb = []
            func_name_emb = []
            byterep = hl_f.replace('.profile','_byterep.pickle') #拿出hl_f所對應的byterep
            rep = pickle.load(open(byterep,'rb')) #list type
            if len(rep) < max_length: #Y1
                for _ in range(max_length-len(rep)):
                    rep.append(0)
            with open(hl_f,encoding='ISO 8859-1') as f: #X2
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
            lines = lines.splitlines()
            if len(lines) > max_length:
                print(hl_f,len(lines))
                break
            for line in lines:
                temp = re.sub(dil," ",line.lower()) # lower? 跟先前一致
                temp = temp.split(" ")
                temp = list(filter(None, temp))
                temp = ' '.join(temp)
                func_name = temp.split(' ')[0] # X1
                if func_name not in api_li:
                    print('=ERROR:=',profile,'=>',temp)
                func_name_emb.append(encode_dict[func_name]) # encode証術數數字
                emb = model.embed_sentence(temp)
                emb = emb[0]
                profile_emb.append(emb) #放入embdding，句子=>profile
            sen2vec_length_normalize.append(len(lines))
            if len(lines) < max_length:
                for _ in range(max_length-len(lines)):
                    profile_emb.append(sent_pad)
            hl_rep.append(rep) #Y1
            all_profiles.append(profile_emb) #X2
            all_api_name.append(func_name_emb) #X1        
    # samples_pid_test = test_pids_df.index.tolist()
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0) #X1
    return np.array(all_api_name) , np.array(all_profiles) , np.array(hl_rep) , sen2vec_length_normalize

In [ ]:
valid_api,valid_emb,valid_rep,valid_len_norm = Sent2vec_emb_dev('./data/tree-rep-profiles-partial/normal/',
                                                 valid_pids_df.index.tolist(),347)
print(valid_api.shape , valid_emb.shape , valid_rep.shape)
test_api,test_emb,test_rep,test_len_norm = Sent2vec_emb_dev('./data/tree-rep-profiles-partial/normal/',
                                                 test_pids_df.index.tolist(),347)
print(test_api.shape , test_emb.shape , test_rep.shape)

valid_api_main,valid_emb_main,valid_rep_main,valid_len_norm_main = Sent2vec_emb_dev('./data/tree-rep-profiles-partial/normal/',
                                                 valid_main_df.index.tolist(),347)
print(valid_api_main.shape , valid_emb_main.shape , valid_rep_main.shape)
test_api_main,test_emb_main,test_rep_main,test_len_norm_main = Sent2vec_emb_dev('./data/tree-rep-profiles-partial/normal/',
                                                 test_main_df.index.tolist(),347)
print(test_api_main.shape , test_emb_main.shape , test_rep_main.shape)

## stage2
* X1 X2 ; Y2

In [ ]:
train_pids_df,valid_pids_df,test_pids_df = pickle.load(file=open('./data/tree-rep-profiles-partial/pids_3df.pkl','rb'))
train_main_df,valid_main_df,test_main_df = pickle.load(file=open('./data/tree-rep-profiles-partial/main_3df.pkl','rb'))


In [40]:
# def Sent2vec_emb_s2(root_dir,all_df,max_length,model=model,dim=768,encode_dict=encode_dict):
root_dir = './data/tree-rep-profiles-partial/normal/'
all_df = train_pids_df
max_length = 347
dim = 768

samples_pid_dev = all_df.index.tolist()
sent_pad = [0]*dim
all_profiles = []
all_api_name = []
all_fam_ans = []
all_byterep=[]
sen2vec_length_normalize = []
sen2vec_dim_normalize = []

for pid in tqdm(samples_pid_dev):
    profile_paths = glob.glob(root_dir + '*/*/'+pid+'.profile')
    hl_f = profile_paths[0]
    profile_emb = []
    func_name_emb = []
    fam_ans = all_df.loc[pid].values.tolist()
    with open(hl_f,encoding='ISO 8859-1') as f: #X2
        lines = f.read()
    lines = re.sub(r'[^\x00-\x7F]+','', lines)
    lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
    lines = lines.splitlines()
    if len(lines) > max_length:
        print(hl_f,len(lines))
        break
    for line in lines:
#             print(len(lines))
        temp = re.sub(dil," ",line.lower()) # lower? 跟先前一致
        temp = temp.split(" ")
        temp = list(filter(None, temp))
        temp = ' '.join(temp)
        func_name = temp.split(' ')[0] # X1
        if func_name.startswith('winh'):
            print(hl_f,func_name)
        if func_name not in api_li:
            print('=ERROR:=',profile,'=>',temp)
        func_name_emb.append(encode_dict[func_name]) # encode証術數數字
        emb = model.embed_sentence(temp)
        emb = emb[0]
        profile_emb.append(emb) #放入embdding，句子=>profile
        sen2vec_dim_normalize.append(emb)
    sen2vec_length_normalize.append(len(lines))
    if len(lines) < max_length:
        for _ in range(max_length-len(lines)):
            profile_emb.append(sent_pad)
    all_profiles.append(profile_emb) #X2
    all_api_name.append(func_name_emb) #X1
    all_fam_ans.append(fam_ans) #Y2
all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0) #X1
#     return np.array(all_api_name) , np.array(all_profiles) , np.array(all_fam_ans) , np.array(sen2vec_dim_normalize),sen2vec_length_normalize


  6%|▌         | 386/6251 [00:28<07:04, 13.82it/s]


 13%|█▎        | 821/6251 [00:55<04:11, 21.62it/s]


 20%|██        | 1271/6251 [01:19<04:20, 19.09it/s]


 28%|██▊       | 1720/6251 [01:42<03:35, 20.99it/s]


 35%|███▌      | 2193/6251 [02:04<02:55, 23.11it/s]


...

 72%|███████▏  | 4493/6251 [03:53<01:28, 19.77it/s]


 79%|███████▉  | 4957/6251 [04:15<00:58, 22.24it/s]


 87%|████████▋ | 5424/6251 [04:36<00:46, 17.95it/s]


 94%|█████████▍| 5903/6251 [04:57<00:19, 18.00it/s]


100%|██████████| 6251/6251 [05:12<00:00, 19.99it/s]

In [ ]:
train_fam_api,train_fam_emb, train_fam_ans, train_normalize , train_fam_len_norm =   Sent2vec_emb_s2('./data/tree-rep-profiles-partial/normal/',train_pids_df,347)
print(train_fam_api.shape,train_fam_emb.shape,train_fam_ans.shape)
valid_fam_api,valid_fam_emb, valid_fam_ans, valid_normalize , valid_fam_len_norm =   Sent2vec_emb_s2('./data/tree-rep-profiles-partial/normal/',valid_pids_df,347)
print(valid_fam_api.shape,valid_fam_emb.shape,valid_fam_ans.shape)
test_fam_api,test_fam_emb, test_fam_ans, test_normalize , test_fam_len_norm =   Sent2vec_emb_s2('./data/tree-rep-profiles-partial/normal/',test_pids_df,347)
print(test_fam_api.shape,test_fam_emb.shape,test_fam_ans.shape)
train_fam_api_main,train_fam_emb_main, train_fam_ans_main, train_normalize_main , train_fam_len_norm_main =   Sent2vec_emb_s2('./data/tree-rep-profiles-partial/normal/',train_main_df,347)
print(train_fam_api_main.shape,train_fam_emb_main.shape,train_fam_ans_main.shape)
valid_fam_api_main,valid_fam_emb_main, valid_fam_ans_main, valid_normalize_main , valid_fam_len_norm_main =   Sent2vec_emb_s2('./data/tree-rep-profiles-partial/normal/',valid_main_df,347)
print(valid_fam_api_main.shape,valid_fam_emb_main.shape,valid_fam_ans_main.shape)
test_fam_api_main,test_fam_emb_main, test_fam_ans_main, test_normalize_main , test_fam_len_norm_main =   Sent2vec_emb_s2('./data/tree-rep-profiles-partial/normal/',test_main_df,347)
print(test_fam_api_main.shape,test_fam_emb_main.shape,test_fam_ans_main.shape)



## dim normalize

In [ ]:
alls = np.concatenate([train_normalize,valid_normalize,test_normalize])
mean = np.mean(alls,axis=0)
std = np.std(alls,axis=0)
mean.shape , std.shape

In [ ]:
def normalize_dim(emb,length,mean=mean,std=std):
    sent_emb = (emb[:length,:] - mean)/std
    final_emb = np.concatenate([sent_emb, emb[length:,:]],axis=0)
    return final_emb

stage 1: pids

In [ ]:
test_emb_norm = []
valid_emb_norm = []
train_emb_norm = []
for emb,length in zip(train_emb,train_len_norm):
    emb_norm = normalize_dim(emb,length)
    train_emb_norm.append(emb_norm)
for emb,length in zip(valid_emb,valid_len_norm):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_emb,test_len_norm):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)
train_emb = np.array(train_emb_norm)
valid_emb = np.array(valid_emb_norm)
test_emb = np.array(test_emb_norm)
print(train_emb.shape,valid_emb.shape,test_emb.shape)

stage 2 : pids

In [ ]:
test_emb_norm = []
valid_emb_norm = []
train_emb_norm = []
for emb,length in zip(train_fam_emb,train_fam_len_norm):
    emb_norm = normalize_dim(emb,length)
    train_emb_norm.append(emb_norm)
for emb,length in zip(valid_fam_emb,valid_fam_len_norm):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_fam_emb,test_fam_len_norm):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)
train_fam_emb = np.array(train_emb_norm)
valid_fam_emb = np.array(valid_emb_norm)
test_fam_emb = np.array(test_emb_norm)
print(train_fam_emb.shape,valid_fam_emb.shape,test_fam_emb.shape)

In [ ]:
pickle.dump(file=open('data/tree-rep-profiles-partial/pids_stage1_train.pkl','wb')
            ,obj=(train_api , train_emb , train_rep),protocol=4)
pickle.dump(file=open('data/tree-rep-profiles-partial/pids_stage1_valid.pkl','wb')
            ,obj=(valid_api,valid_emb,valid_rep),protocol=4)
pickle.dump(file=open('data/tree-rep-profiles-partial/pids_stage1_test.pkl','wb')
            ,obj=(test_api,test_emb,test_rep),protocol=4)

pickle.dump(file=open('data/tree-rep-profiles-partial/pids_stage2_train.pkl','wb')
            ,obj=(train_fam_api,train_fam_emb, train_fam_ans),protocol=4)
pickle.dump(file=open('data/tree-rep-profiles-partial/pids_stage2_valid.pkl','wb')
            ,obj=(valid_fam_api,valid_fam_emb, valid_fam_ans),protocol=4)
pickle.dump(file=open('data/tree-rep-profiles-partial/pids_stage2_test.pkl','wb')
            ,obj=(test_fam_api,test_fam_emb, test_fam_ans),protocol=4)

pickle.dump(file=open('data/tree-rep-profiles-partial/pids_norm_mean_std.pkl','wb'),protocol=4,obj=(mean,std))

In [ ]:
alls = np.concatenate([train_normalize_main,valid_normalize_main,test_normalize_main])
mean = np.mean(alls,axis=0)
std = np.std(alls,axis=0)
mean.shape , std.shape

In [ ]:
def normalize_dim(emb,length,mean=mean,std=std):
    sent_emb = (emb[:length,:] - mean)/std
    final_emb = np.concatenate([sent_emb, emb[length:,:]],axis=0)
    return final_emb

stage 1: main

In [ ]:
test_emb_norm = []
valid_emb_norm = []
train_emb_norm = []
for emb,length in zip(train_emb_main,train_len_norm_main):
    emb_norm = normalize_dim(emb,length)
    train_emb_norm.append(emb_norm)
for emb,length in zip(valid_emb_main,valid_len_norm_main):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_emb_main,test_len_norm_main):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)
train_emb_main = np.array(train_emb_norm)
valid_emb_main = np.array(valid_emb_norm)
test_emb_main = np.array(test_emb_norm)
print(train_emb_main.shape,valid_emb_main.shape,test_emb_main.shape)

stage 2 : main

In [ ]:
test_emb_norm = []
valid_emb_norm = []
train_emb_norm = []
for emb,length in zip(train_fam_emb_main,train_fam_len_norm_main):
    emb_norm = normalize_dim(emb,length)
    train_emb_norm.append(emb_norm)
for emb,length in zip(valid_fam_emb_main,valid_fam_len_norm_main):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_fam_emb_main,test_fam_len_norm_main):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)
train_fam_emb_main = np.array(train_emb_norm)
valid_fam_emb_main = np.array(valid_emb_norm)
test_fam_emb_main = np.array(test_emb_norm)
print(train_fam_emb_main.shape,valid_fam_emb_main.shape,test_fam_emb_main.shape)

In [ ]:
pickle.dump(file=open('data/tree-rep-profiles-partial/main_stage1_train.pkl','wb')
            ,obj=(train_api_main , train_emb_main , train_rep_main),protocol=4)
pickle.dump(file=open('data/tree-rep-profiles-partial/main_stage1_valid.pkl','wb')
            ,obj=(valid_api_main,valid_emb_main,valid_rep_main),protocol=4)
pickle.dump(file=open('data/tree-rep-profiles-partial/main_stage1_test.pkl','wb')
            ,obj=(test_api_main,test_emb_main,test_rep_main),protocol=4)

pickle.dump(file=open('data/tree-rep-profiles-partial/main_stage2_train.pkl','wb')
            ,obj=(train_fam_api_main,train_fam_emb_main, train_fam_ans_main),protocol=4)
pickle.dump(file=open('data/tree-rep-profiles-partial/main_stage2_valid.pkl','wb')
            ,obj=(valid_fam_api_main,valid_fam_emb_main, valid_fam_ans_main),protocol=4)
pickle.dump(file=open('data/tree-rep-profiles-partial/main_stage2_test.pkl','wb')
            ,obj=(test_fam_api_main,test_fam_emb_main, test_fam_ans_main),protocol=4)

pickle.dump(file=open('data/tree-rep-profiles-partial/main_norm_mean_std.pkl','wb'),protocol=4,obj=(mean,std))

In [64]:
std

array([0.10334578, 0.11405497, 0.06538235, 0.06785391, 0.06254525,
       0.06345211, 0.08668455, 0.10869598, 0.03971416, 0.06496501,
       0.06663771, 0.05273993, 0.04326721, 0.05416039, 0.07038648,
       0.05967322, 0.05875815, 0.07865535, 0.04446603, 0.05894471,
       0.05925903, 0.04750017, 0.06672404, 0.05946211, 0.06008697,
       0.06524453, 0.05604865, 0.04845351, 0.05454576, 0.07140453,
       0.07407025, 0.05640128, 0.04082104, 0.06392868, 0.06409959,
       0.04364156, 0.0660302 , 0.0934603 , 0.05111779, 0.05198821,
       0.07442261, 0.04427066, 0.06302553, 0.05363473, 0.06668412,
       0.06243694, 0.04889592, 0.05620024, 0.06822288, 0.04432672,
       0.06005287, 0.0611467 , 0.05240953, 0.05541139, 0.06070184,
       0.03652696, 0.04232733, 0.04098164, 0.06960042, 0.05983368,
       0.04726802, 0.07457776, 0.06681709, 0.0773292 , 0.05999846,
       0.06569882, 0.06069652, 0.06439168, 0.11419737, 0.05604852,
       0.03668654, 0.05515704, 0.05810484, 0.05617372, 0.08384

# Deprecated

In [4]:
# lower = True:

encode_dict_lower = {}
for i,v in encode_dict.items():
    encode_dict_lower[i.lower()] = v
encode_dict_lower

{'loadlibrary': 1,
 'createprocess': 2,
 'openprocess': 3,
 'exitprocess': 4,
 'terminateprocess': 5,
 'winexec': 6,
 'createremotethread': 7,
 'createthread': 8,
 'copyfile': 9,
 'createfile': 10,
 'deletefile': 11,
 'regsetvalue': 12,
 'regcreatekey': 13,
 'regdeletekey': 14,
 'regdeletevalue': 15,
 'regqueryvalue': 16,
 'regenumvalue': 17,
 'winhttpconnect': 18,
 'winhttpopen': 19,
 'winhttpopenrequest': 20,
 'winhttpreaddata': 21,
 'winhttpsendrequest': 22,
 'internetopen': 23,
 'internetconnect': 24,
 'httpsendrequest': 25,
 'geturlcacheentryinfo': 26}

In [5]:
root_dir = './data/tree-rep-profiles-partial/normal/'
av_col = all_df.columns[1:].tolist()
fam_dir = next(os.walk(root_dir))[1]
fam_dir = [x.split('_')[0] for x in fam_dir]
av_df_col = []
for name in av_col:
    if name in fam_dir:
        av_df_col.append(name)
# av_df_col = ['profile'] + av_df_col
print(len(av_df_col))
av_df_col

44


['3.kazy',
 '4.zbot',
 '6.allaple',
 '11.virtob',
 '12.loadmoney',
 '13.graftor',
 '14.vobfus',
 '17.expiro',
 '18.sytro',
 '21.ramnit',
 '22.domaiq',
 '27.softpulse',
 '29.delf',
 '30.mplug',
 '32.browsefox',
 '33.autoit',
 '37.parite',
 '38.msil',
 '41.outbrowse',
 '42.sirefef',
 '43.mira',
 '44.screensaver',
 '45.firseria',
 '47.somoto',
 '51.shodi',
 '54.soltern',
 '55.zygug',
 '59.megasearch',
 '60.conjar',
 '63.jorik',
 '64.installerex',
 '65.shipup',
 '66.bdmj',
 '71.eggnog',
 '75.fesber',
 '78.vilsel',
 '81.picsys',
 '84.directdow',
 '88.clickdownload',
 '103.yantai',
 '104.mresmon',
 '111.vbran',
 '124.avmh',
 '151.downloadadmin']

In [6]:
def create_fam_data(root_dir,samples_pid,av_df_col=av_df_col):
# root_dir = './data/tree-rep-profiles-partial/normal/'
# samples_pid = samples_pid_train
    name_df = pd.DataFrame(0,columns=av_df_col,index=samples_pid)
    for pid in tqdm(samples_pid):
        paths = glob.glob(root_dir +'*/*/'+pid )
        name_li = []
        for path in paths:
            name_li.append(path.split('/')[4].split('_')[0])
        name_df.loc[pid,list(set(name_li))] = 1
    return name_df

In [7]:
samples_pid_train = pickle.load(file=open('./data/tree-rep-profiles-partial/TRAIN_pidNames.pkl','rb'))
samples_pid_valid = pickle.load(file=open('./data/tree-rep-profiles-partial/DEV_pidNames.pkl','rb'))
samples_pid_test = pickle.load(file=open('./data/tree-rep-profiles-partial/TEST_pidNames.pkl','rb'))


In [8]:
# root_dir = './data/tree-rep-profiles-partial/normal/'
# train_fam_df = create_fam_data(root_dir,samples_pid_train)
# root_dir = './data/tree-rep-profiles-partial/DEV/'
# valid_fam_df = create_fam_data(root_dir,samples_pid_valid)
# root_dir = './data/tree-rep-profiles-partial/TEST/'
# test_fam_df = create_fam_data(root_dir,samples_pid_test)
train_fam_df, valid_fam_df,test_fam_df = pickle.load(open('data/tree-rep-profiles-partial/fam_df.pkl','rb'))

100%|██████████| 264/264 [00:00<00:00, 301.65it/s]


In [9]:
def byterep_ans(byterep_list,max_length):
    '''
    Input: byte sequenxe (list of pickles path) 
    '''
    rep_list=[]
    for byterep in byterep_list:
        rep = pickle.load(open(byterep,'rb'))
        if len(rep) < max_length:
            for _ in range(max_length-len(rep)):
                rep.append(0)
        rep_list.append(rep)
    return rep_list

# 寫不同模組 來處理x2 y1 y2
def Sent2vec_emb(root_dir,all_df,max_length,model=model,dim=768,encode_dict=encode_dict,av_df_col=av_df_col):
    '''
    Create sent2vec profile embedding
    Input: root_dir=>fam-tree-profile
    Output: 
    '''
    sent_pad = [0]*dim
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    sen2vec_dim_normalize=[]
    sen2vec_length_normalize = []
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            byterep_list = [x.split('.')[0]+'_byterep.pickle' for x in hl_list]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            byterep_list = [os.path.join(in_directory, f) for f in byterep_list]
            rep_list = byterep_ans(byterep_list,max_length)
            for profile in hl_list:
                pid = profile.split('/')[-1]#.split('.')[0]
#                 profile_df = all_df[all_df.profile == pid]
                profile_df = all_df[all_df.index == pid]
                fam_ans = list(profile_df.values[0])
#                 fam_ans = list(profile_df[av_df_col].values[0])

                profile_emb = []
                func_name_emb = []
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line.lower()) # lower? 跟先前一致
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    func_name = temp.split(' ')[0]
                    if func_name not in api_li:
                        print('=ERROR:=',profile,'=>',temp)
                    func_name_emb.append(encode_dict[func_name])
                    emb = model.embed_sentence(temp)
                    emb = emb[0]
                    profile_emb.append(emb)
                    sen2vec_dim_normalize.append(emb)
                    sen2vec_length_normalize.append(len(lines))
                if len(lines) < max_length:
                    for _ in range(max_length-len(lines)):
                        profile_emb.append(sent_pad)
                all_profiles.append(profile_emb)
                all_api_name.append(func_name_emb)
                all_fam_ans.append(fam_ans)
            all_byterep.extend(rep_list)
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0)
    return np.array(all_profiles) , np.array(all_api_name) , np.array(all_fam_ans) , np.array(all_byterep),np.array(sen2vec_dim_normalize),sen2vec_length_normalize

In [10]:
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" #等號、底線被保留，注意要跟先前一致
max_length = 213

root_dir = './data/tree-rep-profiles-partial/normal/'
train_emb, train_emb_api, train_fam_ans, train_rep_ans, train_norm,train_norm_len = Sent2vec_emb(root_dir,train_fam_df,max_length,encode_dict=encode_dict_lower)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)

root_dir = './data/tree-rep-profiles-partial/DEV/'
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans, valid_norm,valid_norm_len  = Sent2vec_emb(root_dir,valid_fam_df,max_length,encode_dict=encode_dict_lower)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)

root_dir = './data/tree-rep-profiles-partial/TEST/'
test_emb, test_emb_api,test_fam_ans,test_rep_ans, test_norm,test_norm_len = Sent2vec_emb(root_dir,test_fam_df,max_length,encode_dict=encode_dict_lower)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

for val in test_emb:
    if len(val) != max_length:
        print(len(val))

  0%|          | 0/44 [00:00<?, ?it/s]

train of sent2vec vector: (11141, 213, 768) (11141, 213) (11141, 44) (11141, 213)


  2%|▏         | 1/44 [00:00<00:06,  7.04it/s]

valid of sent2vec vector: (437, 213, 768) (437, 213) (437, 44) (437, 213)


100%|██████████| 44/44 [00:07<00:00,  4.48it/s]


test of sent2vec vector: (424, 213, 768) (424, 213) (424, 44) (424, 213)


## normalize dim-wise

In [11]:
alls = np.concatenate([test_norm,valid_norm,train_norm])
mean = np.mean(alls,axis=0)
std = np.std(alls,axis=0)
mean.shape , std.shape

((768,), (768,))

In [12]:
def normalize_dim(emb,length,mean=mean,std=std):
    sent_emb = (emb[:length,:] - mean)/std
    final_emb = np.concatenate([sent_emb, emb[length:,:]],axis=0)
    return final_emb

In [13]:
test_emb_norm = []
valid_emb_norm = []
train_emb_norm = []
for emb,length in zip(train_emb,train_norm_len):
    emb_norm = normalize_dim(emb,length)
    train_emb_norm.append(emb_norm)
for emb,length in zip(valid_emb,valid_norm_len):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_emb,test_norm_len):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)

In [14]:
train_emb = np.array(train_emb_norm)
valid_emb = np.array(valid_emb_norm)
test_emb = np.array(test_emb_norm)
print(train_emb.shape,valid_emb.shape,test_emb.shape)

(11141, 213, 768) (437, 213, 768) (424, 213, 768)


In [15]:
pickle.dump(file=open('data/tree-rep-profiles-partial/TRAIN_sent2vec_vec.pkl','wb'),protocol=4,obj=(train_emb, train_emb_api, train_fam_ans, train_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/DEV_sent2vec_vec.pkl','wb'),protocol=4,obj=(valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/TEST_sent2vec_vec.pkl','wb'),protocol=4,obj=(test_emb, test_emb_api,test_fam_ans,test_rep_ans))
# pickle.dump(file=open('data/tree-rep-profiles-partial/fam_df.pkl','wb'),protocol=4,obj=(train_fam_df, valid_fam_df,test_fam_df))
pickle.dump(file=open('data/tree-rep-profiles-partial/norm_mean_std.pkl','wb'),protocol=4,obj=(mean,std))